## Basic TF-IDF

In [2]:
# run this statement only once to install Rake
!pip install rake_nltk

zsh:1: command not found: pip


In [3]:
import numpy as np
import pandas as pd
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import re, nltk, spacy, gensim
from nltk.tokenize import ToktokTokenizer
from nltk.stem import wordnet
from nltk.corpus import stopwords
from string import punctuation

import pandas as pd
import glob


import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re

### Step 1: Read in and analyse the data

In [4]:


path = r'./dataSmall/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)



# df = pd.read_csv('Kickstarter057.csv')

df = frame
df

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,34,Find out what happens when Albert decides to p...,"{""id"":46,""name"":""Children's Books"",""slug"":""pub...",843,US,the United States,1564969508,"{""id"":1185753820,""name"":""Kelly Airhart"",""slug""...",USD,$,...,alberts-cookie-a-childrens-book,https://www.kickstarter.com/discover/categorie...,True,False,successful,1567632714,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",843.000000,international
1,25,Following the success of our Hastings and St L...,"{""id"":349,""name"":""Letterpress"",""slug"":""publish...",873,GB,the United Kingdom,1498480246,"{""id"":2133715710,""name"":""EEK Publications"",""sl...",GBP,£,...,sneeky-peek-presents-a-picturesque-guide-book-...,https://www.kickstarter.com/discover/categorie...,False,False,failed,1507617030,1.319919,"{""web"":{""project"":""https://www.kickstarter.com...",876.426502,international
2,8,Join us on this amazing journey to impacting 1...,"{""id"":49,""name"":""Periodicals"",""slug"":""publishi...",234,AU,Australia,1437294575,"{""id"":124148187,""name"":""Brenda van Rensburg"",""...",AUD,$,...,reaching-the-stars-with-spacetripreneurs,https://www.kickstarter.com/discover/categorie...,False,False,failed,1439893575,0.737314,"{""web"":{""project"":""https://www.kickstarter.com...",234.465725,international
3,512,An anthology of queer witchery.,"{""id"":324,""name"":""Anthologies"",""slug"":""publish...",18131,US,the United States,1605839241,"{""id"":154688021,""name"":""dave ring"",""slug"":""dav...",USD,$,...,unfettered-hexes-queer-tales-of-insatiable-dar...,https://www.kickstarter.com/discover/categorie...,True,True,successful,1609509126,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",18131.000000,international
4,0,"Written by a chronically ill girl, commenting ...","{""id"":50,""name"":""Poetry"",""slug"":""publishing/po...",0,US,the United States,1462053258,"{""id"":514222826,""name"":""Moriah Rabeaux"",""is_re...",USD,$,...,walking-into-winter-writing-to-warm-the-frostb...,https://www.kickstarter.com/discover/categorie...,False,False,failed,1464816474,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",0.000000,international
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14492,4,"Inspired by Burning Man, I am building a water...","{""id"":258,""name"":""Architecture"",""slug"":""design...",65,US,the United States,1405433682,"{""id"":1449913350,""name"":""Mike Warren"",""slug"":""...",USD,$,...,bug-free-monkey-hut,https://www.kickstarter.com/discover/categorie...,False,False,failed,1408030621,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",65.000000,international
14493,124,International collaboration by Yoshiko Chuma &...,"{""id"":6,""name"":""Dance"",""slug"":""dance"",""positio...",11462,US,the United States,1328645073,"{""id"":1403167449,""name"":""Yoshiko Chuma"",""is_re...",USD,$,...,love-story-palestine,https://www.kickstarter.com/discover/categorie...,True,False,successful,1335931141,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",11462.450000,international
14494,57,'STRUM' A vibrant dance performance celebratin...,"{""id"":254,""name"":""Performances"",""slug"":""dance/...",4154,US,the United States,1391542191,"{""id"":1928397609,""name"":""UPside Dance Company""...",USD,$,...,upside-dance-presents-strum-with-composer-mark...,https://www.kickstarter.com/discover/categorie...,True,True,successful,1396421940,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",4154.000000,international
14495,48,I want to support people that are experiencing...,"{""id"":343,""name"":""Candles"",""slug"":""crafts/cand...",2116,GB,the United Kingdom,1591768463,"{""id"":1366582721,""name"":""Jessica Cronshaw"",""sl..

In [5]:
def extract_cat(text):
    text = text.split(",")
    
    text = text[1] + " " +text[2]
    

    text = text.replace ("/", " ")
    
    
    text = text.replace ("name", "")
    text = text.replace ("slug", "")
    
    text = text.replace ('"', "")
    text = text.replace ('{', "")
    text = text.replace (':', "")
    
    text = text.lower()
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text) # matches all whitespace characters
    text = text.strip(' ')
    return text



df['category'] = df['category'].apply(lambda x: extract_cat(x))


df.head()



,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,34,Find out what happens when Albert decides to p...,children's books publishing children's books,843,US,the United States,1564969508,"{""id"":1185753820,""name"":""Kelly Airhart"",""slug""...",USD,$,...,alberts-cookie-a-childrens-book,https://www.kickstarter.com/discover/categorie...,True,False,successful,1567632714,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",843.000000,international
1,25,Following the success of our Hastings and St L...,letterpress publishing letterpress,873,GB,the United Kingdom,1498480246,"{""id"":2133715710,""name"":""EEK Publications"",""sl...",GBP,£,...,sneeky-peek-presents-a-picturesque-guide-book-...,https://www.kickstarter.com/discover/categorie...,False,False,failed,1507617030,1.319919,"{""web"":{""project"":""https://www.kickstarter.com...",876.426502,international
2,8,Join us on this amazing journey to impacting 1...,periodicals publishing periodicals,234,AU,Australia,1437294575,"{""id"":124148187,""name"":""Brenda van Rensburg"",""...",AUD,$,...,reaching-the-stars-with-spacetripreneurs,https://www.kickstarter.com/discover/categorie...,False,False,failed,1439893575,0.737314,"{""web"":{""project"":""https://www.kickstarter.com...",234.465725,international
3,512,An anthology of queer witchery.,anthologies publishing anthologies,18131,US,the United States,1605839241,"{""id"":154688021,""name"":""dave ring"",""slug"":""dav...",USD,$,...,unfettered-hexes-queer-tales-of-insatiable-dar...,https://www.kickstarter.com/discover/categorie...,True,True,successful,1609509126,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",18131.000000,international
4,0,"Written by a chronically ill girl, commenting ...",poetry publishing poetry,0,US,the United States,1462053258,"{""id"":514222826,""name"":""Moriah Rabeaux"",""is_re...",USD,$,...,walking-into-winter-writing-to-warm-the-frostb...,https://www.kickstarter.com/discover/categorie...,False,False,failed,1464816474,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",0.000000,international


In [6]:

df = df[['name','category','blurb']]
df.head()



,name,category,blurb
0,Albert's Cookie- A Children's Book,children's books publishing children's books,Find out what happens when Albert decides to p...
1,SNEEKY PEEK presents a Picturesque Guide Book ...,letterpress publishing letterpress,Following the success of our Hastings and St L...
2,REACHING the STARS with SpaceTripreneurs,periodicals publishing periodicals,Join us on this amazing journey to impacting 1...
3,Unfettered Hexes: Queer Tales of Insatiable Da...,anthologies publishing anthologies,An anthology of queer witchery.
4,Walking Into Winter: Writing to Warm the Frost...,poetry publishing poetry,"Written by a chronically ill girl, commenting ..."


### Step 2a: Data pre-processing

In [7]:

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


df['blurb']=df['blurb'].map(lambda s:preprocess(s)) 
    

<ipython-input-7-44050645d60f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['blurb']=df['blurb'].map(lambda s:preprocess(s))


In [8]:
df.head()

,name,category,blurb
0,Albert's Cookie- A Children's Book,children's books publishing children's books,find happens albert decides play food takes hi...
1,SNEEKY PEEK presents a Picturesque Guide Book ...,letterpress publishing letterpress,following success hastings leonard guide book ...
2,REACHING the STARS with SpaceTripreneurs,periodicals publishing periodicals,join amazing journey impacting million children
3,Unfettered Hexes: Queer Tales of Insatiable Da...,anthologies publishing anthologies,anthology queer witchery
4,Walking Into Winter: Writing to Warm the Frost...,poetry publishing poetry,written chronically ill girl commenting ups do...


In [9]:
df['category_processed'] = ""


for index, row in df.iterrows():
    cat = row['category']
    
    # instantiating a Rake object
    # by default it uses english stopwords from NLTK (natural language tool kit)
    # and discards all puntuation characters
    r = Rake()

    # extracting the keywords from the text by passing plot 
    r.extract_keywords_from_text(cat)

    # getting the dictionary whith key words and their scores
    key_words_dict_scorestwo = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['category_processed'] = [w for w in key_words_dict_scorestwo.keys()]

<ipython-input-9-0a6530d97d4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category_processed'] = ""


In [10]:
df.head()

,name,category,blurb,category_processed
0,Albert's Cookie- A Children's Book,children's books publishing children's books,find happens albert decides play food takes hi...,"[books, children, publishing]"
1,SNEEKY PEEK presents a Picturesque Guide Book ...,letterpress publishing letterpress,following success hastings leonard guide book ...,"[letterpress, publishing]"
2,REACHING the STARS with SpaceTripreneurs,periodicals publishing periodicals,join amazing journey impacting million children,"[periodicals, publishing]"
3,Unfettered Hexes: Queer Tales of Insatiable Da...,anthologies publishing anthologies,anthology queer witchery,"[anthologies, publishing]"
4,Walking Into Winter: Writing to Warm the Frost...,poetry publishing poetry,written chronically ill girl commenting ups do...,"[poetry, publishing]"


In [11]:
df['blurb_processed'] = ""

for index, row in df.iterrows():
    desc = row['blurb']
    
    # instantiating a Rake object
    # by default it uses english stopwords from NLTK (natural language tool kit)
    # and discards all puntuation characters
    r = Rake()

    # extracting the keywords from the text by passing plot 
    r.extract_keywords_from_text(str(desc))

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    #stemming
#     row['desc_stem'] = [stemmer.stem(w) for w in key_words_dict_scores.keys()]
    row['blurb_processed'] = [w for w in key_words_dict_scores.keys()]
    
df.head()

<ipython-input-11-923fd48c99d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['blurb_processed'] = ""


,name,category,blurb,category_processed,blurb_processed
0,Albert's Cookie- A Children's Book,children's books publishing children's books,find happens albert decides play food takes hi...,"[books, children, publishing]","[find, happens, albert, decides, play, food, t..."
1,SNEEKY PEEK presents a Picturesque Guide Book ...,letterpress publishing letterpress,following success hastings leonard guide book ...,"[letterpress, publishing]","[following, success, hastings, leonard, guide,..."
2,REACHING the STARS with SpaceTripreneurs,periodicals publishing periodicals,join amazing journey impacting million children,"[periodicals, publishing]","[join, amazing, journey, impacting, million, c..."
3,Unfettered Hexes: Queer Tales of Insatiable Da...,anthologies publishing anthologies,anthology queer witchery,"[anthologies, publishing]","[anthology, queer, witchery]"
4,Walking Into Winter: Writing to Warm the Frost...,poetry publishing poetry,written chronically ill girl commenting ups do...,"[poetry, publishing]","[written, chronically, ill, girl, commenting, ..."


In [12]:
df = df[['name', 'category_processed', 'blurb_processed']]

df.head()

,name,category_processed,blurb_processed
0,Albert's Cookie- A Children's Book,"[books, children, publishing]","[find, happens, albert, decides, play, food, t..."
1,SNEEKY PEEK presents a Picturesque Guide Book ...,"[letterpress, publishing]","[following, success, hastings, leonard, guide,..."
2,REACHING the STARS with SpaceTripreneurs,"[periodicals, publishing]","[join, amazing, journey, impacting, million, c..."
3,Unfettered Hexes: Queer Tales of Insatiable Da...,"[anthologies, publishing]","[anthology, queer, witchery]"
4,Walking Into Winter: Writing to Warm the Frost...,"[poetry, publishing]","[written, chronically, ill, girl, commenting, ..."


### Step 3: Create word representation via a bag of words 

Using the values from the df columns

In [13]:
# name should be omitted from bag of words creation
columns = df.columns[1:]


df['bag_of_words'] = ''

for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'bag_of_words':
            # to convert the list into a string of words separated by a space
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words

# let's keep only the title and the bag of words in the dataframe
df.head()


,name,category_processed,blurb_processed,bag_of_words
0,Albert's Cookie- A Children's Book,"[books, children, publishing]","[find, happens, albert, decides, play, food, t...",books children publishing find happens albert ...
1,SNEEKY PEEK presents a Picturesque Guide Book ...,"[letterpress, publishing]","[following, success, hastings, leonard, guide,...",letterpress publishing following success hasti...
2,REACHING the STARS with SpaceTripreneurs,"[periodicals, publishing]","[join, amazing, journey, impacting, million, c...",periodicals publishing join amazing journey im...
3,Unfettered Hexes: Queer Tales of Insatiable Da...,"[anthologies, publishing]","[anthology, queer, witchery]",anthologies publishing anthology queer witchery
4,Walking Into Winter: Writing to Warm the Frost...,"[poetry, publishing]","[written, chronically, ill, girl, commenting, ...",poetry publishing written chronically ill girl...


In [14]:
df.drop(columns = ['category_processed'], inplace = True)
df.drop(columns = ['blurb_processed'], inplace = True)
df.head()

,name,bag_of_words
0,Albert's Cookie- A Children's Book,books children publishing find happens albert ...
1,SNEEKY PEEK presents a Picturesque Guide Book ...,letterpress publishing following success hasti...
2,REACHING the STARS with SpaceTripreneurs,periodicals publishing join amazing journey im...
3,Unfettered Hexes: Queer Tales of Insatiable Da...,anthologies publishing anthology queer witchery
4,Walking Into Winter: Writing to Warm the Frost...,poetry publishing written chronically ill girl...


### Step 4: Create cosine similarity matrix 

In [19]:
# instantiating and generating the count matrix using CountVectorizer
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

In [20]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.0745356  0.18257419 ... 0.         0.         0.06666667]
 [0.0745356  1.         0.10206207 ... 0.         0.         0.0745356 ]
 [0.18257419 0.10206207 1.         ... 0.         0.         0.09128709]
 ...
 [0.         0.         0.         ... 1.         0.         0.35805744]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.06666667 0.0745356  0.09128709 ... 0.35805744 0.         1.        ]]


### Step 5: Create and Run the model (recommender)

In [21]:
# creating a Series for the movie titles and corresponding indexes
indices = pd.Series(df['name'])
indices[:5]
df = df[['name', 'bag_of_words']]

df.head()

,name,bag_of_words
0,Albert's Cookie- A Children's Book,books children publishing find happens albert ...
1,SNEEKY PEEK presents a Picturesque Guide Book ...,letterpress publishing following success hasti...
2,REACHING the STARS with SpaceTripreneurs,periodicals publishing join amazing journey im...
3,Unfettered Hexes: Queer Tales of Insatiable Da...,anthologies publishing anthology queer witchery
4,Walking Into Winter: Writing to Warm the Frost...,poetry publishing written chronically ill girl...


In [22]:
def recommendations(name, cosine_sim = cosine_sim):
    
    recommended = []
    
    # getting the index of the movie with the given title
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:12].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended.append(list(df['name'])[i])
        
    return recommended

In [26]:
recommendations("Albert's Cookie- A Children's Book")

['Alex and Sally See the Galaxy - a picture book',
 'Cats, Dogs and Books',
 "Vlad's First Fang",
 'Oh Mummy, why can’t we go out and play?',
 'Tastes of Lunar New Year',
 'The "Mr. Baker\'s Book" Series',
 'Mustapha Panda',
 'Joaquin and the Pekkybirds, A Brooklyn Story',
 "Children's Book Series",
 'If I Could, I Would...',
 'The Adventures of Mr. Schnoozle']

In [27]:
text = "Albert's Cookie- A Children's Book"

df.loc[df['name'] == text]

,name,bag_of_words
0,Albert's Cookie- A Children's Book,books children publishing find happens albert decides play food takes high flying adventure outer space


In [28]:
id = 0
pd.set_option("display.max_colwidth", None)
frame[id:id+1]

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,34,"Find out what happens when Albert decides to play with his food, and takes it on a high flying adventure to outer space.",children's books publishing children's books,843,US,the United States,1564969508,"{""id"":1185753820,""name"":""Kelly Airhart"",""slug"":""kellyairhart"",""is_registered"":null,""is_email_verified"":null,""chosen_currency"":null,""is_superbacker"":null,""avatar"":{""thumb"":""https://ksr-ugc.imgix.net/assets/007/555/634/d22975a4b83632a44e53e4839109188d_original.jpg?ixlib=rb-2.1.0&w=40&h=40&fit=crop&v=1564969450&auto=format&frame=1&q=92&s=6830845015cc6cb047686bf4ea6f7aab"",""small"":""https://ksr-ugc.imgix.net/assets/007/555/634/d22975a4b83632a44e53e4839109188d_original.jpg?ixlib=rb-2.1.0&w=80&h=80&fit=crop&v=1564969450&auto=format&frame=1&q=92&s=7955e25504864216ee9360be95557ecd"",""medium"":""https://ksr-ugc.imgix.net/assets/007/555/634/d22975a4b83632a44e53e4839109188d_original.jpg?ixlib=rb-2.1.0&w=160&h=160&fit=crop&v=1564969450&auto=format&frame=1&q=92&s=b4a0e34e7f541d3e2af6ef97519bf948""},""urls"":{""web"":{""user"":""https://www.kickstarter.com/profile/kellyairhart""},""api"":{""user"":""https://api.kickstarter.com/v1/users/1185753820?signature=1616130838.503d1c9611fd987ff31145220aaa919b176da153""}}}",USD,$,...,alberts-cookie-a-childrens-book,https://www.kickstarter.com/discover/categories/publishing/children's%20books,True,False,successful,1567632714,1.0,"{""web"":{""project"":""https://www.kickstarter.com/projects/kellyairhart/alberts-cookie-a-childrens-book?ref=discovery_category_newest"",""rewards"":""https://www.kickstarter.com/projects/kellyairhart/alberts-cookie-a-childrens-book/rewards""}}",843.0,international
